In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# File Path
data = pd.read_excel(r"INSERT HERE")

# Columns
columns_of_interest = [f"X{i}" for i in range(1, 28)] + ["status", "industry"]
model_data = data[columns_of_interest]

# Prod vs. Serv
production_industries = ['A', 'C', 'F']
service_industries = ['G', 'H', 'I', 'J', 'L', 'M', 'N', 'P', 'Q', 'R', 'S']

# Evaluation function
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_proba)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    return accuracy, auc_score, conf_matrix, class_report

# SVM
svm_model_prod = make_pipeline(StandardScaler(), SVC(kernel='linear', C=100, gamma='scale', probability=True))
svm_model_serv = make_pipeline(StandardScaler(), SVC(kernel='linear', C=100, gamma='scale', probability=True))
svm_model_prod.fit(X_train_prod, y_train_prod)
svm_model_serv.fit(X_train_serv, y_train_serv)

# Predict
industries = model_data['industry'].unique()
for industry in industries:
    industry_data = model_data[model_data['industry'] == industry]
    X_ind = industry_data.drop(['status', 'industry'], axis=1)
    y_ind = industry_data['status']
    X_train_ind, X_test_ind, y_train_ind, y_test_ind = train_test_split(X_ind, y_ind, test_size=0.30, random_state=99, stratify=y_ind)
    
    if industry in production_industries:
        model = svm_model_prod
    else:
        model = svm_model_serv

    #Results
    accuracy_ind, auc_score_ind, conf_matrix_ind, class_report_ind = evaluate_model(model, X_test_ind, y_test_ind)
    print(f"\nResults for Industry {industry}:")
    print("Accuracy:", accuracy_ind)
    print("AUC Score:", auc_score_ind)